<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/GEMINI_AGENT_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import os
from google import genai
from google.genai import types
from google.genai.errors import APIError

# --- IMPORTANT SETUP (API KEY RETRIEVAL) ---
# This block is essential for retrieving the key from your Colab Secrets.

# Default to None, then check environment variables
GOOGLE_API_KEY = os.environ.get('GOOGLE_API_KEY') or os.environ.get('GEMINI_API_KEY')

# Check for Colab-specific userdata
try:
    from google.colab import userdata
    # Assumes your key is saved in Colab Secrets under the name 'GEMINI'
    key_from_colab = userdata.get('GEMINI')
    if key_from_colab:
        GOOGLE_API_KEY = key_from_colab
        print("API Key retrieved from Colab Secrets 'GEMINI'.")
except ImportError:
    print("Not in Colab. Using environment variable for API Key.")

# --- 1. LLM CONFIGURATION & API CLIENT ---

class AgentConfig:
    LLM_MODEL_NAME: str = "gemini-2.5-flash"
    SYSTEM_PROMPT: str = (
        "You are an expert Synergized RAG Agent. Your task is to solve complex, multi-step questions "
        "by strictly following a Thought-Action-Observation loop. "
        "You must analyze the initial query and the current context (History) to decide your next move. "
        "If you need to search for information, your response MUST be formatted as: "
        "\nThought: [Your reasoning on the next step, based on the History]"
        "\nAction: SEARCH([The specific fact or query to look up])"
        "\nIf you have sufficient information to answer the original question, your response MUST be formatted as: "
        "\nThought: [Your concluding thoughts and final answer generation]"
        "\nAction: FINISH([The final answer to the original question])"
        "\nDO NOT include any text outside the Thought: and Action: tags in your final response."
    )

# Initialize the Gemini Client
if GOOGLE_API_KEY:
    try:
        client = genai.Client(api_key=GOOGLE_API_KEY)
        client_initialized = True
        print(f"Gemini Client ready with model: {AgentConfig.LLM_MODEL_NAME}")
    except Exception as e:
        client = None
        client_initialized = False
        print(f"Error initializing Gemini Client: {e}")
else:
    client = None
    client_initialized = False
    print("FATAL: GOOGLE_API_KEY is None. Client cannot be initialized.")


# --- 2. KNOWLEDGE BASE (RETRIEVER TOOL) ---

knowledge_base = {
    "CEO of Tesla": "Elon Musk",
    "Company founded by Elon Musk in 2002": "SpaceX",
    "Primary product of SpaceX": "Rockets and spacecraft",
}

def retrieve_fact(query: str) -> str:
    """Simulates the retrieval tool (the 'Action' fulfillment)."""
    # Normalize query by stripping quotes and spaces for exact match
    cleaned_query = query.strip().strip('"').strip("'")
    fact = knowledge_base.get(cleaned_query)

    if fact:
        return fact
    else:
        return f"Not found: no exact match for '{cleaned_query}'"

# --- 3. CORE LLM INTEGRATION FUNCTION (CORRECTED) ---

def call_llm_agent(prompt: str) -> str:
    """
    Makes the actual API call to the Gemini LLM for Thought and Action.
    FIX: Passes the prompt directly as 'contents' and uses 'system_instruction'
         in config to avoid all role/part argument errors.
    """
    if not client:
        return "Error: Gemini Client not initialized. Cannot make API call."

    try:
        response = client.models.generate_content(
            model=AgentConfig.LLM_MODEL_NAME,
            contents=prompt, # <-- FIX: Pass the prompt string directly.
            config=types.GenerateContentConfig(
                system_instruction=AgentConfig.SYSTEM_PROMPT, # <-- FIX: Pass system instruction here
                temperature=0.0,
                max_output_tokens=512
            )
        )
        return response.text
    except APIError as e:
        # Pass the full error message for debugging
        return f"Error during Gemini API call: {e}"
    except Exception as e:
        return f"An unexpected error occurred: {e}"

# --- 4. SYNERGIZED RAG AGENT LOOP ---

def rag_agent_loop_live(initial_query: str, max_steps: int = 5):
    """Orchestrates the iterative RAG ⇌ Reasoning loop."""
    if not client_initialized:
        print("\nDemo failed: Gemini client is not available. Please check your API key setup.")
        return

    print("\n" + "="*80)
    print(f"--- Synergized RAG Agent with LIVE LLM Integration ---")
    print(f"Initial Query: {initial_query}")
    print("="*80)

    current_context = f"Original Query: {initial_query}\n\n---\nHistory:\n"

    for step in range(1, max_steps + 1):
        print(f"\n--- STEP {step} / {max_steps} ---")

        # 1. LLM REASONING & ACTION SELECTION
        llm_prompt = f"{current_context}\n\nWhat is the next Thought and Action? (Step {step})"
        llm_response_text = call_llm_agent(llm_prompt)

        # --- PARSING LLM OUTPUT ---

        # Check for API/Fatal error first
        if "Error during Gemini API call" in llm_response_text or "An unexpected error occurred" in llm_response_text:
             print(f"FATAL API ERROR: {llm_response_text}. Terminating.")
             return

        # Parse Thought and Action using regex
        # Use re.DOTALL to allow matching across newlines
        thought_match = re.search(r"Thought: (.*?)(\nAction:|$)", llm_response_text, re.DOTALL)
        action_match = re.search(r"Action: (.*)", llm_response_text, re.DOTALL)

        thought = thought_match.group(1).strip() if thought_match else f"ERROR: Thought not parsed from LLM response:\n{llm_response_text}"
        action_command = action_match.group(1).strip() if action_match else f"ERROR: Action not parsed from LLM response:\n{llm_response_text}"

        print(f"  [PARSED THOUGHT] {thought}")
        print(f"  [PARSED ACTION] {action_command}")

        # 2. EXECUTE ACTION

        if action_command.startswith("SEARCH"):
            search_query_match = re.search(r"SEARCH\((.*)\)", action_command)
            search_query = search_query_match.group(1).strip() if search_query_match else "ERROR: Search query not parsed."

            # Execute Retrieval (the RAG component)
            retrieved_data = retrieve_fact(search_query)

            # 3. CONTEXT INTEGRATION (OBSERVATION)
            observation = f"Observation: Retrieved fact for '{search_query}' is '{retrieved_data}'"
            print(f"  [{observation}]")

            # Update the context for the next LLM turn
            current_context += f"Step {step}:\nThought: {thought}\nAction: {action_command}\n{observation}\n"

        elif action_command.startswith("FINISH"):
            final_answer_match = re.search(r"FINISH\((.*)\)", action_command)
            final_answer = final_answer_match.group(1).strip() if final_answer_match else "ERROR: Final answer not parsed."

            print("\n" + "="*80)
            print("--- FINAL RESULT ---")
            print(f"Final Answer: {final_answer}")
            print("\n--- Full Context History ---")
            print(current_context)
            print("="*80)
            return

        else:
            # Catch cases where the model hallucinates a non-FINISH/non-SEARCH action
            print(f"\nFATAL ERROR: Unknown or malformed Action: {action_command}. Terminating.")
            print("LLM failed to adhere to the required format.")
            return

    print("\n--- LOOP TERMINATED ---")
    print("Max steps reached without a FINISH action. Agent failed to complete the task.")

# --- EXECUTION ---
if client_initialized:
    query = "What is the primary product of the company founded by the CEO of Tesla?"
    rag_agent_loop_live(query)
else:
    print("\n*** The live demo cannot run because the Gemini Client failed to initialize. ***")

API Key retrieved from Colab Secrets 'GEMINI'.
Gemini Client ready with model: gemini-2.5-flash

--- Synergized RAG Agent with LIVE LLM Integration ---
Initial Query: What is the primary product of the company founded by the CEO of Tesla?

--- STEP 1 / 5 ---
  [PARSED THOUGHT] The first step is to identify the CEO of Tesla. This information is crucial to proceed with the rest of the query.
  [PARSED ACTION] SEARCH(CEO of Tesla)
  [Observation: Retrieved fact for 'CEO of Tesla' is 'Elon Musk']

--- STEP 2 / 5 ---
  [PARSED THOUGHT] I have identified Elon Musk as the CEO of Tesla. The next step is to find out which company Elon Musk founded, as the query asks for the primary product of "the company founded by the CEO of Tesla." I should search for companies founded by Elon Musk.
  [PARSED ACTION] SEARCH(companies founded by Elon Musk)
  [Observation: Retrieved fact for 'companies founded by Elon Musk' is 'Not found: no exact match for 'companies founded by Elon Musk'']

--- STEP 3 / 5 --